# What to do?

Process will look like this:
1. Implement/extend generator
2. Think of possible errors/bugs errors introducer can introduce to existing samples
3. Save samples with introduced bugs as separate dataset (to consider)
4. Let model learn on generated correct samples
5. Let previously learnt model learn on expanded bugged dataset. This dataset will contain samples paired/connected with bugged samples. In this process correct sample will be provided together with bugged samples (or all bugged samples connected with correct sample) and during training bugged sample will be treated as noised sample and goal of denoising will be to get correct sample back. HAVE TO MODIFIE MODEL FOR IT OR ADD ANOTHER METHOD THAT WILL TRAIN IN THIS WAY OR MAYBE STEERE TRAIN_STEP USING PARAMETER DURING CONSTRUCTION

## introduction of 'errors' end their repairs (v1)

First version of generator outputs simple samples so errors are also probably simple. Thus only those 3 (but really 2 cause first two are the same but different symbols)

Errors:
* randomly remove operations tokens (mathematical operations) or equal signs or semicolons and shift other tokens by amount of tokens removed. ***"Ponieważ ta wersja języka jest bardzo prosta to ciężko wymyślić bardziej skomplikwoane błędy do dodania. Należy również zauważyć, że według próbke pomiędzy kolejnymi ID lub literałami (NUM, STRING) powinny znajdować się znaki równości, operacje matematyczne lub średniki. Więc jeżeli model jest w stanie naprawić próbkę, która została zniekształcona za pomocą jednej z powyższych operacji, to istnieje potencjał wykorzystania tej metody w bardziej skomplikowanych przypadkach błędów czy też języków"***

* ~~(there are no functions in this version so maybe not here) add parenthesis after ID that is not a function~~

In [ ]:
single_sample = ['char','ID0','(','int','ID1',',','char','ID2',',','char','ID3',')','{','int','ID4',';',
'char','ID5',';','printf','(','STRING',',','ID2',')',';','char','ID6','=','ID2','/','ID3',
'+','ID2','/','ID0','-','STRING',';','ID0','=','ID3','/','ID6','*','ID3','+','STRING',
'*','ID3','+','STRING',';','int', 'ID7', '=', 'NUM', ';', 'ID1', '=', 'ID7', '-', 'ID4', '-',
'ID4','/','ID7','+','NUM',';','int','ID8',';','printf','(','STRING',',','ID8',')',
';','int','ID9','=','ID8','*','ID8','/','ID7','-','ID7','*','NUM',';','return',
'NUM',';','}','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY',
'EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY',
'EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY',
'EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY',
'EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY',
'EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY',
'EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY',
'EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY',
'EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY',
'EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY',
'EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY',
'EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY','EMPTY',
'EMPTY','EMPTY','EMPTY','EMPTY','EMPTY']
print(len(single_sample))
print(" ".join(single_sample).replace(";", ";\n").replace("{", "{\n").replace("EMPTY", ""))

### vvv This cell is going to go entirely into separate py file so it can be reused vvv

In [ ]:
import random
import numpy as np

In [ ]:
# sample is a sequence (list of tokens) of generated/loaded code
def remove_token_and_shift_sample_randomized(tokens_to_remove, chance_to_survive):
  def apply(sample):
    to_return = [token for token in sample if (token not in tokens_to_remove or random.random() < chance_to_survive)]
    return to_return + ["EMPTY" for _ in range(len(sample) - len(to_return))]
  return apply
  
def remove_token_and_shift_sample(tokens_to_remove):
  def apply(sample):
    to_return = [token for token in sample if (token not in tokens_to_remove)]
    return to_return + ["EMPTY" for _ in range(len(sample) - len(to_return))]
  return apply

class ErrorsIntroducer:
  def __init__(self, errors_introducers) -> None:
    self.errors_introducers = errors_introducers

  '''
  no_of_bugged_samples - should be used only in case introducers are based on some kind of randomness and You want to have multiple takes on the same 
  '''
  def apply(self, sample, no_of_takes):
    if not no_of_takes or no_of_takes <= 0:
      no_of_takes = 1
    bugged_sample = sample
    samples = []
    for _ in range(no_of_takes):
      for introducer in self.errors_introducers:
        bugged_sample = introducer(bugged_sample)
      samples.append(bugged_sample)
    return samples

In [ ]:
introducer = ErrorsIntroducer([remove_token_and_shift_sample_randomized([";", "+", "-", "/", "="], 0.5)])
out = introducer.apply(single_sample, 5)
print(len(out))
for each in out:
  print(" ".join(each).replace(";", ";\n").replace("{", "{\n").replace("EMPTY", ""))

In [ ]:
from simple_c_v1_lang_generator import fill_vocabulary_c_v1, vocabulary_c_v1
fill_vocabulary_c_v1()
cv1Dict = {el:idx for idx,el in enumerate(vocabulary_c_v1)}
def tokens_to_vals(sample):
  return [cv1Dict[token] for token in sample]

# load original sample

# saving errors
original_data_dir_path = "..\\data\\testu_testu"
bugged_data_dir_path = original_data_dir_path + "_bugged"
connection_file_path = bugged_data_dir_path + "\\connections_to_original_index"
connections = {}

final_dataset = np.asarray([tokens_to_vals(single_sample)])
print(final_dataset.shape)
for idx, sample in enumerate(final_dataset):
  sample_filename = f"sample_no{idx}.txt"
  np.savetxt(f"{original_data_dir_path}\\{sample_filename}", sample, newline=" ", fmt="%u")

  bugged_samples = introducer.apply(sample, 5)
  current_connections = []
  for no_of_bugged_sample, bugged_sample in enumerate(bugged_samples):
    filename = f"sample_no{idx}_take{no_of_bugged_sample}.txt"
    current_connections.append(filename)
    np.savetxt(f"{bugged_data_dir_path}\\{filename}", bugged_sample, newline=" ", fmt="%u")
  connections[sample_filename] = current_connections
# save connections files
with open(connection_file_path, 'w') as connectionsFileHandler:
  for key in connections:
    print(connections[key])
    connectionsFileHandler.write(",".join([key, *connections[key]]) + "\n")

In [ ]:
# loading 
with open(connection_file_path, 'r') as connectionsFileHandler:
  lines = connectionsFileHandler.readlines()
  lines = [line.strip().split(',') for line in lines]
  print(lines)
  original = lines[0][0]
  print(original)
  bugged = lines[0][1:]
  print(bugged)

### Version for numpy

In [ ]:
sample_vals = tokens_to_vals(single_sample)

In [ ]:
# sample is a sequence (list of tokens) of generated/loaded code
def remove_token_and_shift_sample_randomized(tokens_to_remove, chance_to_survive, dictionary, sample_len):
  EMPTY = dictionary["EMPTY"]
  tokens_to_remove = [dictionary[token] for token in tokens_to_remove]
  def apply(samples):
    to_return = [[token for token in sample if (token not in tokens_to_remove or random.random() < chance_to_survive)] for sample in samples]
    to_return = [sample + [EMPTY for _ in range(sample_len - len(sample))] for sample in to_return]
    return to_return
  return apply

class ErrorsIntroducer:
  def __init__(self, errors_introducers) -> None:
    self.errors_introducers = errors_introducers

  '''
  no_of_bugged_samples - should be used only in case introducers are based on some kind of randomness and You want to have multiple takes on the same 
  '''
  def apply(self, samples):
    for introducer in self.errors_introducers:
      samples = introducer(samples)
    return np.asarray(samples)